In [ ]:
#setup requirements
%pip install -r ./../requirements.txt

In [ ]:
# load openai_api_key from .env file
from dotenv import load_dotenv
import os

load_dotenv()
openai_api_key = os.environ.get('OPENAI_API_KEY')

model="gpt-3.5-turbo"

In [ ]:
# import of langchain
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.schema import StrOutputParser

In [ ]:
text_input = """

    # Medienmitteilung

    ## Bundesrat genehmigt generelles Projekt zur Engpassbeseitigung zwischen Genf und Nyon
    Bern, 25.10.2023 - Der Bundesrat hat an seiner Sitzung vom 25.10.2023 das generelle Projekt zur Engpassbeseitigung zwischen der Verzweigung Le Vengeron und dem Anschluss Nyon auf der Autobahn A1 genehmigt. Dank dieser Massnahme kann ein flüssiger und sicherer Verkehrsfluss auf diesem Autobahnabschnitt gewährleistet und die Verkehrsbelastung im nachgelagerten Strassennetz erheblich reduziert werden.

    Die geplanten Arbeiten dienen dazu, den Engpass auf der Autobahn A1 zwischen Genf, Coppet und Nyon zu beseitigen. Ausserdem werden dadurch der Verkehrsablauf auf der Kantonsstrasse sowie die Effizienz der Autobahnanschlüsse verbessert. Um den Bedürfnissen des öffentlichen Regionalverkehrs Rechnung zu tragen, werden Bus-Streifen eingerichtet. Ebenfalls vorgesehen sind gesicherte Infrastrukturen für den Langsamverkehr.

    Die wichtigsten im generellen Projekt vorgesehenen baulichen Massnahmen sind:

    die Verbreiterung der Autobahn auf drei Fahrstreifen pro Fahrtrichtung auf einer Länge von rund 19 Kilometern zwischen der Verzweigung Le Vengeron und dem Anschluss Nyon. Ausserdem wird die Autobahnachse auf einem etwa 2,8 Kilometer langen Abschnitt versetzt, um ein Amphibienlaichgebiet zu erhalten, welches im Inventar der Amphibienlaichgebiete von nationaler Bedeutung verzeichnet ist;
    der Umbau der Verzweigung Le Vengeron samt Bau neuer Zubringer;
    die Umgestaltung des Anschlusses Coppet und
    die Reorganisation des Anschlusses Nyon samt Bau neuer Zubringer.
    Die Kosten des Vorhabens betragen rund 956 Millionen Franken.

    Das Parlament hat dieses Projekt in den Ausbauschritt 2023 für die Nationalstrassen aufgenommen. Dies unter dem Vorbehalt, dass der Bundesrat das generelle Projekt bis spätestens Ende 2023 genehmigt. Diese Voraussetzung ist inzwischen erfüllt. Die vollständige Inbetriebnahme ist für 2041 vorgesehen. Im Jahr 2033 sollen die Bauarbeiten aufgenommen werden.

    Adresse für Rückfragen

    Mediendienst Bundesamt für Strassen ASTRA +41 58 464 14 91
    
"""

In [ ]:
# Analyse der Fakten und Zusammenfassung der Medienmitteilung
prompt = PromptTemplate.from_template("""
    Analysiere den folgenden Text und 
    identifiziere alle wichtigen Fakten.
    Fasse die Medenienmitteilung, mit Fokus auf wichtige Fakten zusammen.
    Der Text ist von news.admin.ch und eine 
    offizielle Medienmitteilung: \n {topic}\n\n
    Gebe folgendes zurück; deine kurze Analyse zu den Fakten und anschliessend die eigentliche medienmitteilung auf das wesentliche zusammengefasst:
""")
runnable = prompt | ChatOpenAI(model=model) | StrOutputParser()
result_analyse = runnable.invoke({"topic": text_input})
print(result_analyse)

In [ ]:
# 10 keywords aus Analyse der Fakten und Zusammenfassung der Medienmitteilung
prompt = PromptTemplate.from_template("""
    Analysiere den folgenden Text und 
    identifiziere 10 Keywords, die gut in Google 
    (deutschsprachige Schweiz) ranken würden. 
    Der Text ist von news.admin.ch und daher eine 
    offizielle Medienmitteilung: \n {topic}\n\n
    Beispiele für Keywords: #EFD #Bildung #Schweiz #Bildungsraum #Bildungsbericht2019 #EDK #WBF #Bildungspolitik #Bildungssystem #Bildungssystem\n\n
    Achte darauf das vor jedem Keyword ein # steht und das das Keyword jeweils nur aus einem Wort besteht. \n\n
    Deine 10 Keywords:#
""")
runnable = prompt | ChatOpenAI(model=model) | StrOutputParser()
result_keywords10 = runnable.invoke({"topic": result_analyse})
print(result_keywords10)

In [ ]:
# 3 gute Keywords aus der Top 10 bzw. der Analyse der Fakten und Zusammenfassung der Medienmitteilung
prompt = PromptTemplate.from_template("""
    Analysiere den folgenden Text und 
    identifiziere exakt 3 Keywords, die gut in Google 
    (deutschsprachige Schweiz) ranken würden. 
    Der Text ist von news.admin.ch und daher eine 
    offizielle Medienmitteilung: \n {topic}\n\n
    Mögliche gute Keywords für diese Medienmitteilung: {keywords10}\n\n
    Achte darauf das vor jedem Keyword ein # steht und das das Keyword jeweils nur aus einem Wort besteht. \n\n
    Deine 3 Keywords:
""")
runnable = prompt | ChatOpenAI(model=model) | StrOutputParser()
result_keywords03 = runnable.invoke({"topic": result_analyse, "keywords10": result_keywords10})
print(result_keywords03)

In [ ]:
# Tweet mit den 3 top Keywords, 3 emojis und einem Link zur Medienmitteilung
prompt = PromptTemplate.from_template("""
    Du bist politisch neutral und erstellst einen Tweet aus einer Medienmitteilung.
    Du arbeitest in der Social Media Abteilung vom Bundesrat. Dein Tweet fasst das
    wichtigste der Medienmitteilung zusammen. Medienmittelung: \n {topic}\n\n
    Es ist wichtig das in deinem Tweet folgende 3 Keywords enthalten sind: {keywords3}\n\n
    Achte darauf das vor jedem Keyword ein # steht und das das Keyword jeweils nur aus einem Wort besteht. \n\n
    Der Tweet endet immer mit [URL-Medienmitteilung]
    Deine politisch neutraler tweet inkl 3 emojis, mit den 3 gewünschten Keywords:
""")
runnable = prompt | ChatOpenAI(model=model) | StrOutputParser()
result = runnable.invoke({"topic": result_analyse, "keywords3": result_keywords03})
print(result)

In [ ]:
# Mögliche Fragen von 20min.ch vom Reporter
prompt = PromptTemplate.from_template("""
    Du bist ein Reporter von 20min.ch und wirst 10 kritische Fragen erfassen.
    Die Fragen sind kritisch, aber nicht beleidigend und beziehen sich auf die Medienmitteilung: \n {topic}\n\n
    Deine 10 kritischen Fragen:
""")
runnable = prompt | ChatOpenAI(model=model) | StrOutputParser()
result_fragen_20min = runnable.invoke({"topic": result_analyse})
print(result_fragen_20min)

In [ ]:
# Mögliche Fragen von der NZZ Reporterin
prompt = PromptTemplate.from_template("""
    Du bist eine Reporterin von der NZZ (Neue Zürcher Zeitung) und wirst 10 kritische Fragen erfassen.
    Die Fragen sind kritisch, aber nicht beleidigend und beziehen sich auf die Medienmitteilung: \n {topic}\n\n
    Deine 10 kritischen Fragen:
""")
runnable = prompt | ChatOpenAI(model=model) | StrOutputParser()
result_fragen_nzz = runnable.invoke({"topic": result_analyse})
print(result_fragen_nzz)

In [ ]:
# Zusammenfassung der Medienmitteilung für ein 10 jähriges Kind
prompt = PromptTemplate.from_template("""
    Erstelle eine Zusammenfassung aus folgendem Text für ein 10 jähriges Kind: {topic} \n\n
    Deine kindergerechte Zusammenfassung:
""")
runnable = prompt | ChatOpenAI(model=model) | StrOutputParser()
result_zusammenfassung_kind = runnable.invoke({"topic": result_analyse})
print(result_zusammenfassung_kind)

In [ ]:
# prompt für Dalle 3
prompt = PromptTemplate.from_template("""
    Erstelle ein Prompt für DALL·E 3 aus folgendem Text: {topic} \n\n
    Prompt für DALL·E 3 als strichzeichnung (comic-style ohne sprechblasen):
""")
runnable = prompt | ChatOpenAI(model=model) | StrOutputParser()
result_dall3prompt= runnable.invoke({"topic": result_zusammenfassung_kind})
print(result_dall3prompt)

In [ ]:
from langchain.utilities.dalle_image_generator import DallEAPIWrapper
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.llms import OpenAI

llm = OpenAI(temperature=0.9)
prompt = PromptTemplate(
    input_variables=["image_desc"],
    template="Generate a detailed prompt based on: {image_desc}",
)
chain = LLMChain(llm=llm, prompt=prompt)
image_url = DallEAPIWrapper().run(chain.run(result_dall3prompt))

In [ ]:
image_url